# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
import matplotlib.pyplot as plt                    # Dependencies and Setup
import pandas as pd
import numpy as np
import requests, json, re
import gmaps
import os

from config import (ow_key, gkey)                  # Import API keys

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import the data from the WeatherPy exercise in csv format

file = "../WeatherPy/output_data/cities_weather_data.csv"
weather_data_df = pd.read_csv(file, encoding="utf-8")

weather_data_df.drop('Degrees From Equator', inplace=True, axis=1)
weather_data_df.head()


,City,Country Code,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date
0,Cherskiy,RU,68.7500,161.3000,-16.0,96.0,32.0,2.64,27/04/21
1,Chivay,PE,-15.6383,-71.6011,13.0,35.0,7.0,1.90,27/04/21
2,Rikitea,PF,-23.1203,-134.9692,25.0,71.0,48.0,3.53,27/04/21
3,Roseburg,US,43.2165,-123.3417,9.0,87.0,90.0,1.54,27/04/21
4,Saint-Georges,GF,12.0564,-61.7485,29.0,74.0,40.0,9.26,27/04/21


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps

gmaps.configure(api_key=gkey)                               # Configure the gkey

locations = weather_data_df[["Latitude", "Longitude"]]      # Store Latitude and Longitude data into locations variable

humidity = weather_data_df["Humidity"]                      # Store Humidity data into humidity variable

In [8]:
# Create the Heatmap layer

fig = gmaps.figure(center=(0.0, 30.0), zoom_level=2)        # Set up the figure

heat_layer = gmaps.heatmap_layer(locations, 
                         weights=humidity,                  # configure the heat layer
                         dissipating=False, 
                         max_intensity=weather_data_df["Humidity"].max(),
                         point_radius = 3)


fig.add_layer(heat_layer)                                   # Add the layer to the figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# Create a dataframe with me ideal vacation locations

ideal_vacay_spots_df = weather_data_df.loc[(weather_data_df['Temperature'] >= 21) &   # Build the dataframe 
                                        (weather_data_df['Temperature'] <= 25) &      # with ideal criteria
                                        (weather_data_df['Wind Speed'] <= 5) & 
                                        (weather_data_df['Cloudiness'] <= 10)
                                        ].reset_index(drop=True)

ideal_vacay_spots_df[['Hotel Name','Country']] = ''               # Add two more columns that will be populated below

ideal_vacay_spots_df

,City,Country Code,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date,Hotel Name,Country
0,Ingham,AU,42.6000,-84.3833,22.0,52.0,1.0,2.57,27/04/21,,
1,Castro,CL,-24.7911,-50.0119,22.0,42.0,1.0,2.79,27/04/21,,
2,Sairang,IN,23.8000,92.6667,25.0,49.0,0.0,1.64,27/04/21,,
3,Komsomolskiy,RU,40.4272,71.7189,24.0,23.0,0.0,1.03,27/04/21,,
4,Saint-Francois,GP,46.4154,3.9054,21.0,32.0,0.0,3.60,27/04/21,,
5,Alanya,TR,36.5438,31.9998,21.0,83.0,0.0,4.63,27/04/21,,
6,Eldorado,BR,-23.7869,-54.2836,24.0,29.0,0.0,3.18,27/04/21,,
7,Graaff-Reinet,ZA,-32.2522,24.5308,21.0,46.0,4.0,0.45,27/04/21,,
8,Moron,MN,-34.6534,-58.6198,22.0,54.0,0.0,1.34,27/04/21,,
9,Kozan,TR,37.4552,35.8157,23.0,56.0,0.0,4.63,27/04/21,,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
#Use the Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in ideal_vacay_spots_df.iterrows():            # For-loop to get data for each ideal location

    params = {
    "location": f"{row[2]},{row[3]}",                         # Gets latitude and longitude from the dataframe above
    "radius": 5000,
    "type": "lodging",
    "keyword" : "hotel",
    "language" : "en",
    "key": gkey
    }
    
    try:
        first_hotel = requests.get(base_url, params=params).json()['results'][0]            # Regex to extract full    
        ideal_vacay_spots_df.at[index, 'Hotel Name'] = first_hotel['name']                  # country name. But it 
        country, = re.findall('\s.*,\s(.*?)$', first_hotel['plus_code']['compound_code'])   # doesn't always work. So
        ideal_vacay_spots_df.at[index, 'Country'] = country                                 # try-except is setup to
    except (KeyError, ValueError):                                                          # leave the country code
                ideal_vacay_spots_df.at[index, 'Country'] = row['Country Code']             # from citypy in these 
    except IndexError : ideal_vacay_spots_df.at[index, 'Hotel Name'] = 'No hotel found'                                                                                       # cases. 

ideal_vacay_spots_df

,City,Country Code,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date,Hotel Name,Country
0,Ingham,AU,42.6000,-84.3833,22.0,52.0,1.0,2.57,27/04/21,No hotel found,
1,Castro,CL,-24.7911,-50.0119,22.0,42.0,1.0,2.79,27/04/21,Hotel Buganville Express,Brazil
2,Sairang,IN,23.8000,92.6667,25.0,49.0,0.0,1.64,27/04/21,No hotel found,
3,Komsomolskiy,RU,40.4272,71.7189,24.0,23.0,0.0,1.03,27/04/21,Diyor hotel,Uzbekistan
4,Saint-Francois,GP,46.4154,3.9054,21.0,32.0,0.0,3.60,27/04/21,No hotel found,
5,Alanya,TR,36.5438,31.9998,21.0,83.0,0.0,4.63,27/04/21,Kahya Hotel,TR
6,Eldorado,BR,-23.7869,-54.2836,24.0,29.0,0.0,3.18,27/04/21,Villa Verde Hotel,Brazil
7,Graaff-Reinet,ZA,-32.2522,24.5308,21.0,46.0,4.0,0.45,27/04/21,Drostdy Hotel,South Africa
8,Moron,MN,-34.6534,-58.6198,22.0,54.0,0.0,1.34,27/04/21,No hotel found,
9,Kozan,TR,37.4552,35.8157,23.0,56.0,0.0,4.63,27/04/21,Yaver'in Konağı,Turkey


In [16]:
# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index,             # A beautiful list comprehension method, 
              row in ideal_vacay_spots_df.iterrows()]                # thanks for the tip

locations = ideal_vacay_spots_df[["Latitude", "Longitude"]]

In [17]:
#Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

markers = gmaps.marker_layer(locations,                              # Set up markers using locations and info_box_content
                             info_box_content= hotel_info)           # variables from previous cell

fig.add_layer(markers)                                               # Add the layer and                                      
fig                                                                  # display the figure 

Figure(layout=FigureLayout(height='420px'))